# Offset

In [17]:
import threading
import queue
import time
import numpy as np
import sounddevice as sd
import serial
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import matplotlib.pyplot as plt
from ordered_set import OrderedSet
import pickle

In [3]:
import sys
sys.path.append('..')
# Custom imports
from analysis_utils import DataContainer
from audio_constants import ref_volume, dB_to_scalar

In [5]:
ESP_COM_PORT = "COM20"
ESP_BAUDRATE = 115200

SLM_COM_PORT = "COM20"
SLM_BAUDRATE = 115200

In [6]:
# Get the default audio playback device
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)

# Create a volume control interface
volume_controller = cast(interface, POINTER(IAudioEndpointVolume))

In [18]:
# Function to receive serial data and store it
def receive_esp_serial_data(queue):
    data = np.array([])
    ser = serial.Serial(ESP_COM_PORT, ESP_BAUDRATE)
    while not finish_flag:
        received_data = ser.readline().decode().strip()
        received_data = received_data.split(" ")[0]
        try:
            received_data = float(received_data)
        except ValueError:
            received_data = float(received_data.split(".")[0])
        try:
            if log_flag:
                data = np.append(data, received_data)
                print(received_data)
        except KeyError:
            pass
    ser.close()
    queue.put(data)

In [ ]:
# Function to receive serial data and store it
def receive_slm_serial_data(queue):
    data = np.array([])
    ser = serial.Serial(SLM_COM_PORT, SLM_BAUDRATE)
    while not finish_flag:
        received_data = ser.readline().decode().strip()
        received_data = received_data.split(" ")[0]
        try:
            received_data = float(received_data)
        except ValueError:
            received_data = float(received_data.split(".")[0])
        try:
            if log_flag:
                data = np.append(data, received_data)
                print(received_data)
        except KeyError:
            pass
    ser.close()
    queue.put(data)

In [8]:
# Function to play audio sine wave
def play_test_wave():
    global log_flag, finish_flag
    duration = 15 if TIME_PERIOD == 0.125 else 30 # s
    sample_rate = 88200
    t = np.arange(0, duration, 1 / sample_rate)

    sine_wave = np.sin(2 * np.pi * 1000 * t)
    volume_controller.SetMasterVolumeLevelScalar(dB_to_scalar[ref_volume], None)

    finish_flag = False
    log_flag = True

    # Wait before playing the sine wave
    time.sleep(0.375 if TIME_PERIOD == 0.125 else 3) #s

    sd.play(sine_wave, sample_rate, blocking=True)

    # Wait after playback is done
    time.sleep(0.375 if TIME_PERIOD == 0.125 else 3) #s

    log_flag = False

    # Send a signal to the other thread to terminate
    finish_flag = True

In [19]:
# Create queues to store received serial data
esp_queue = queue.Queue()
slm_queue = queue.Queue()

# Flag to signal termination
finish_flag = False
# Flag to signal when to log measurements
log_flag = False

## Ejecutar pruebas

### Periodo F, sin ponderación A

In [ ]:
TIME_PERIOD = 0.125

# Create the serial data receiving threads
esp_serial_thread = threading.Thread(target=receive_esp_serial_data, args=(esp_queue,), daemon=True)
slm_serial_thread = threading.Thread(target=receive_slm_serial_data, args=(slm_queue,), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_wave, daemon=True)

# Start serial threads
esp_serial_thread.start()
slm_serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
esp_serial_thread.join()
slm_serial_thread.join()
audio_thread.join()

# Reset the flag for next test
finish_flag = False

# Retrieve data from the thread
offset_data_F_unwted = esp_queue.get()
offset_data_F_unwted_ref = slm_queue.get()

### Periodo S, sin ponderación A

In [ ]:
TIME_PERIOD = 1.0

# Create the serial data receiving thread
esp_serial_thread = threading.Thread(target=receive_esp_serial_data, args=(esp_queue,), daemon=True)
slm_serial_thread = threading.Thread(target=receive_slm_serial_data, args=(slm_queue,), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_wave, daemon=True)

# Start serial thread
esp_serial_thread.start()
slm_serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
esp_serial_thread.join()
slm_serial_thread.join()
audio_thread.join()

# Reset the flag for next test
finish_flag = False

# Retrieve data from the thread
offset_data_S_unwted = esp_queue.get()
offset_data_S_unwted_ref = slm_queue.get()

### Periodo F, con ponderación A

In [ ]:
TIME_PERIOD = 0.125

# Create the serial data receiving thread
esp_serial_thread = threading.Thread(target=receive_esp_serial_data, args=(esp_queue,), daemon=True)
slm_serial_thread = threading.Thread(target=receive_slm_serial_data, args=(slm_queue,), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_wave, daemon=True)

# Start serial thread
esp_serial_thread.start()
slm_serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
esp_serial_thread.join()
slm_serial_thread.join()
audio_thread.join()

# Reset the flag for next test
finish_flag = False

# Retrieve data from the thread
offset_data_F_wted = esp_queue.get()
offset_data_F_wted_ref = slm_queue.get()

### Periodo S, con ponderación A

In [ ]:
TIME_PERIOD = 1.0

# Create the serial data receiving thread
esp_serial_thread = threading.Thread(target=receive_esp_serial_data, args=(esp_queue,), daemon=True)
slm_serial_thread = threading.Thread(target=receive_slm_serial_data, args=(slm_queue,), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_wave, daemon=True)

# Start serial thread
esp_serial_thread.start()
slm_serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
esp_serial_thread.join()
slm_serial_thread.join()
audio_thread.join()

# Reset the flag for next test
finish_flag = False

# Retrieve data from the thread
offset_data_S_wted = esp_queue.get()
offset_data_S_wted_ref = slm_queue.get()

## Guardar datos

In [9]:
with open("../results/offset/F_unwted.pkl", "wb") as file:
    pickle.dump(offset_data_F_unwted, file)

with open("../results/offset/S_unwted.pkl", "wb") as file:
    pickle.dump(offset_data_S_unwted, file)

with open("../results/offset/F_wted.pkl", "wb") as file:
    pickle.dump(offset_data_F_wted, file)

with open("../results/offset/S_wted.pkl", "wb") as file:
    pickle.dump(offset_data_S_wted, file)

In [ ]:
with open("../results/offset/F_unwted_ref.pkl", "wb") as file:
    pickle.dump(offset_data_F_unwted_ref, file)

with open("../results/offset/S_unwted_ref.pkl", "wb") as file:
    pickle.dump(offset_data_S_unwted_ref, file)

with open("../results/offset/F_wted_ref.pkl", "wb") as file:
    pickle.dump(offset_data_F_wted_ref, file)

with open("../results/offset/S_wted_ref.pkl", "wb") as file:
    pickle.dump(offset_data_S_wted_ref, file)